In [2]:
# ----------------------------------------------------------------------------------
# Group Members:
# Chew Hong Ern
# Irdina Syaurah Binte Mohd Azrol
# Shannon Phang Jia Min
# Tan Youyi
# ----------------------------------------------------------------------------------





# ------------ import modules and set \n (for formatting in f-strings) -------------
import json
import textwrap 
nl = "\n"
nl10 = "\n          | "
nl15 = "\n             | "
nl40 = "\n                                      | "
# ----------------------------------------------------------------------------------




# --------------- Function to sort and display papers (used later) -----------------
def sort_and_display_papers(match = []):
    while True:
        print('''\n\033[1mSort by:\033[0m
Title:     1
Year:      2''')
        sort_by = input("Enter: ")
        if sort_by == "1" or sort_by == "2":
            break
        else:
            print("You have entered an unexpected input. Please try again.")

    if sort_by == "1":
        while True:
            print('''\n\033[1mAccording to:\033[0m
A to Z:    1
Z to A:    2''')
            according1 = input("Enter: ")
            if according1 == "1":
                match.sort(key = lambda x:x["Title"])
                break
            elif according1 == "2":
                match.sort(key = lambda x:x["Title"], reverse = True)
                break
            else:
                print("You have entered an unexpected input. Please try again.")

    elif sort_by == "2":
        while True:
            print('''\n\033[1mAccording to:\033[0m
Oldest to Newest:    1
Newest to Oldest:    2''')
            according2 = input("Enter: ")
            if according2 == "1":
                match.sort(key = lambda x:x["Year"])
                break
            elif according2 == "2":
                match.sort(key = lambda x:x["Year"], reverse = True)
                break
            else:
                print("You have entered an unexpected input. Please try again.")
                
    # finally display all matching papers based on selected sorting
    print(f"{nl}{len(match)} matching results found.")
    for paper in match: 
        print()
        for i in range(len(paper)):
            print(f"{list(paper.keys())[i] : <10}| {textwrap.fill(text=list(paper.values())[i], width = 100).replace(nl, nl10)}")
    print("\nSearch complete.")     
    return
# ----------------------------------------------------------------------------------




# --------------------------------- Search engine ----------------------------------
def final_search_engine():
    # import and open the dataset
    f1 = open("master.json", encoding = "utf - 8")
    v1 = json.loads(f1.read())
    
    # ------------- reformatting the dataset for easier referral later -------------
    papers_info_list = []
    for papers in v1:
        smalllistauthor = []                                   # smalllistauthor is a list of authors for each paper
        for everyauthor in papers["authors"]:                  # combining every author's first and last name to form their fullname
            fullname = f"{everyauthor['fname']} {everyauthor['lname']}"
            smalllistauthor.append(fullname)
        authors = ", ".join(smalllistauthor)                   # similar to allkeyword, combine multiple strings in a list into one string to remove the ''
        
        # transforming all keywords to be titled
        if len(papers["paper"]["keywords"])>0:
            Tkeyword = []
            for words in papers["paper"]["keywords"]:
                Tkeyword.append(words.title())                 
        else:
            Tkeyword = papers["paper"]["keywords"]
        allkeyword = ", ".join(Tkeyword)                       # allkeyword combines multiple strings in a list into one string using join, this is done just to remove the ''
       
        # creating a new dictionary with some of the reformatted data
        paper_info = {                                                  
            "Paper ID": str(papers['paper_id']),
            "Authors": authors,
            "Title": papers['paper']['title'].title(),
            "Year": str(papers['year']),
            "Journal": papers['source']['journal'].title(),
        }
        
    # ---- this section deals with some incomplete/funny inputs from the dataset ----
        # contact authors and emails
        cauthors = []
        cauthors_email = []    
        for no_allauthors in range(len(papers["authors"])):             # loop all authors to find cauthors
            try:
                if papers["authors"][no_allauthors]["cauthor"] == True:
                    cauthor = papers['authors'][no_allauthors]['fname'] + " " + papers['authors'][no_allauthors]['lname']
                    cauthors.append(cauthor)
                    cauthors_email.append(papers['authors'][no_allauthors]['email'])
            except KeyError:
                pass                                                    # some cauthors have no emails which results in KeyError

        if cauthors == []:                                              # if there are no cauthors
            cauthors.append("No contact author found")
            cauthors_email.append("No contact email found")
        paper_info["CAuthor"] = ', '.join(cauthors)
        paper_info["Email"] = ', '.join(cauthors_email)

        # if there are no keywords
        if allkeyword == '':
            paper_info["Keywords"] = "No keywords found"
        else: 
            paper_info["Keywords"] = allkeyword

        # issues with source volume
        try:
            if papers['source']['volume'] == [] or papers['source']['volume'] == False:   # from the original json
                paper_info["Volume"] = "No volume found"                                  # from the new dictionary variable created, append new key ["Volume"]
            else:
                paper_info["Volume"] = str(papers['source']['volume'])
        except KeyError:
            paper_info["Volume"] = "No volume found"                                      # some of the entries yielded KeyErrors

        # issues with source number 
        try:
            if papers['source']['number'] == [] or papers['source']['number'] == False:
                paper_info["Number"] = "No number found"            
            else:
                if type(papers['source']['number']) == list:
                    paper_info["Number"] = ', '.join(map(str, papers['source']['number'])) # paper 209 source number is a list, so we convert it into a neat string
                else:
                    paper_info["Number"] = str(papers['source']['number'])
        except KeyError:
            paper_info["Number"] = "No number found"                                       # some of the entries yielded KeyErrors

        # append pages here for a nice dictionary order
        paper_info["Pages"] = f"{papers['source']['pages']['s_page']}-{papers['source']['pages']['e_page']}"  

    # -- append the whole paper_info dictionary to a list, so that we have a neat database to search through --
        papers_info_list.append(paper_info)          

        
        
    def search_by_authors():
        # ------------- get user to input author name they want to search  --------------
        print("\n\033[1mEnter the name of author to start search.\033[0m")
        search = input("Enter author name: ")

        # --------------------- check if there are matching results ---------------------
        # if the input matches the author in the database (not case-sensitive), then retrieve the dictionary of the paper info
        matching_papers = [info for info in papers_info_list if search.lower() in info["Authors"].lower()]

        # -------------------------------------------------------------------------------
        # The info here is each and every dictionary in our list of dictionaries
        # print every appropriate paper info into a list, as a compilation of all wanted papers, same as:
        # for info in papers_info_list if search.lower() in info["Authors"].lower():
        #       matching_papers.append(info)
        # -------------------------------------------------------------------------------
        
        # if there are no matching results, inform the user
        # otherwise, give user options for sorting by Title (alphabetical order) or Year (chronological order)
        if len(matching_papers) == 0: 
            print(f"\nNo matching papers found for search term '{search}'.\n\nSearch complete.")
        else:
            sort_and_display_papers(match = matching_papers)
        return
    
    
    def search_by_title():
        # ---------- get user to input article title name they want to search  ----------
        print("\n\033[1mEnter the article title to start search.\033[0m")
        search = input("Enter article title: ")
        
        # --------------------- check if there are matching results ---------------------
        matching_papers = [info for info in papers_info_list if search.lower() in info["Title"].lower()]

        if len(matching_papers) == 0:
            print(f"\nNo matching papers found for search term '{search}'.\n\nSearch complete.")
        else:
            sort_and_display_papers(match = matching_papers)
        return
    
    
    def search_papers_by_year():
        # ------------- get user to input specific year they want to search -------------
        print('''\n\033[1mEnter the article year to start search.\033[0m''' )
        # inputs that are not numbers, not 4 digits or in database range are rejected
        while True:
            search = input("\nEnter article year: ")
            for eachcharacter in search:
                if eachcharacter not in "0123456789":
                    flag = "N"
                    print("\nYou have entered an unexpected input. Please try again.=")
                    break
                elif len(search) != 4:
                    flag = "N"
                    print("\nYou need to enter exactly 4 digits. Please try again.")
                    break
                elif int(search)<1994 or int(search)>2021:
                    flag = "N"
                    print("\nYear is not in database range. Please try again.")
                    break
                else:
                    flag = "Y"
            if flag == "Y":
                break   
        
        # --------------------- check if there are matching results ---------------------
        matching_papers = [info for info in papers_info_list if search == info["Year"]]

        if matching_papers:
            # if there are matching results, give user options for sorting by Title (alphabetical order)
            # any unwanted input (not 1 or 2) is rejected using a while loop
            # did not use sort_and_display() function as it would be redundant to sort by year
            while True:
                print('''\n\033[1mSort according to:\033[0m
        A to Z:    1
        Z to A:    2''')
                according1 = input("Enter: ")
                if according1 == "1":
                    matching_papers.sort(key = lambda x:x["Title"])
                    break
                elif according1 == "2":
                    matching_papers.sort(key = lambda x:x["Title"], reverse = True)
                    break
                else:
                    print("\nYou have entered an unexpected input. Please try again.")
                
            # finally display all matching papers based on selected sorting 
            print(f"{nl}{len(matching_papers)} matching results found.")
            for paper in matching_papers: 
                print()
                for i in range(len(paper)):
                    print(f"{list(paper.keys())[i] : <10}| {textwrap.fill(text=list(paper.values())[i], width = 100).replace(nl, nl10)}")
            print("\nSearch complete.")
          
        # inform the user if there are no matches found
        else:
            print(f"\nNo matching papers found for year '{search}'.\n\nSearch complete.")
        return


    def search_papers_by_range():
        # ------------ get user to input range of years they want to search  ------------
        print("\n\033[1mEnter the start and end years to start search.\033[0m")
        while True:
            year1 = input("Enter start year: ") 
            year2 = input("Enter end year: ")

            # inputs that are not numbers, not 4 digits, in database range or have incorrect order are rejected
            for eachcharacter_by_range1, eachcharacter_by_range2 in zip(year1, year2):
                if eachcharacter_by_range1 not in "0123456789" or eachcharacter_by_range2 not in "0123456789":
                    flag = "N"
                    print("\nYou have entered an unexpected input. Please try again.")
                    break
                elif len(year1) != 4 or len(year2) != 4:
                    flag = "N"
                    print("\nYou need to enter exactly 4 digits for each year. Please try again.")
                    break
                elif int(year1)<1994 or int(year1)>2021 or int(year2)<1994 or int(year2)>2021:
                    flag = "N"
                    print("\nYear input is not in database range. Please try again.")
                    break
                elif int(year2)<int(year1):                # additional check if end year is later than start year
                    print("\nStart year is later than end year. Please try again.")
                    break
                else:
                    flag = "Y"      
            if flag == "Y":
                break

        # --------------------- check if there are matching results ---------------------
        categories = list(range(int(year1), int(year2) + 1))
        matching_papers = [info for info in papers_info_list if int(info["Year"]) in categories]
            
        if len(matching_papers) == 0:
            print(f"\nNo matching papers from {year1} to {year2}.\n\nSearch complete.")
        else:
            sort_and_display_papers(match = matching_papers)
        return


    def search_by_journal():
        # ----------- get user to input journal title they want to search ---------------
        print("\n\033[1mEnter the journal title to start search.\033[0m")
        search = input("Enter journal title: ")
        
        # --------------------- check if there are matching results ---------------------
        matching_papers = [info for info in papers_info_list if search.lower() in info["Journal"].lower()]

        if len(matching_papers) == 0:
            print(f"\nNo matching papers found for search term '{search}'.\n\nSearch complete.")
        else:
            sort_and_display_papers(match = matching_papers)
        return
    

    def search_cowritten_papers():
        # ------------ get user to input range of years they want to search ------------
        print("\n\033[1mEnter the name of authors to start search.\033[0m")
        search1 = input("Enter first author: ")
        search2 = input("Enter second author: ")
        
        # --------------------- check if there are matching results ---------------------
        matching_papers = [info for info in papers_info_list if search1.lower() in info["Authors"].lower() and search2.lower() in info["Authors"].lower()]

        if len(matching_papers) == 0:
            print(f"\nNo matching papers found cowritten by '{search1}' and '{search2}'.\n\nSearch complete.")
        else:
            sort_and_display_papers(match = matching_papers)
        return
    
    
    def search_paper_ID():
        # ---------------- get user to input Paper ID they want to search ---------------
        print("\n\033[1mEnter Paper ID to start search.\033[0m")
        
        # inputs that are not numbers or beyond database range is rejected 
        while True:
            search = input("\nEnter Paper ID: ")
            for eachcharacter in search:
                if eachcharacter not in "0123456789":
                    flag = "N"
                    print("\nYou have entered an unexpected input. Please try again.=")
                    break
                elif int(search)<1 or int(search)>216:
                    flag = "N"
                    print("\nPaper ID is not in database range. Please try again.")
                    break
                else:
                    flag = "Y"
            if flag == "Y":
                break   
                           
        # --------------------- check if there are matching results ---------------------
        matching_papers = [info for info in papers_info_list if search == info["Paper ID"]]
                
        # display matching paper
        for paper in matching_papers: 
            print()
            for i in range(len(paper)):
                print(f"{list(paper.keys())[i] : <10}| {textwrap.fill(text=list(paper.values())[i], width = 100).replace(nl, nl10)}")
        print("\nSearch complete.")

        return
    
    def search_gen():
        # ------- get user to input general search term -> article/keyword/journal -------
        print("\n\033[1mEnter search term to start search.\033[0m")
        search = input("Enter search term: ")
        
       # --------------------- check if there are matching results ---------------------
        matching_papers = [info for info in papers_info_list if search.lower() in info["Journal"].lower() or \
                           search.lower() in info["Title"].lower() or search.lower() in info["Keywords"].lower()]

        if len(matching_papers) == 0:
            print(f"\nNo matching papers found for search term '{search}'.\n\nSearch complete.")
        else:
            sort_and_display_papers(match = matching_papers)
        
        return

    
    def this_is_the_search_engine():
    # ------------------------- Combine all the search functions ------------------------
        print('''\033[1m
What would you like to search?
Please enter the corresponding number:\033[0m''')
        print('''
    Authors:            1
    Title:              2
    Year:               3
    Journal:            4
    Co-written papers:  5
    Paper ID:           6
    General:            7
            ''')
        
        # while Loop to contain incorrect inputs
        while True:
            search_term2 = input("\nEnter search term: ")

            if search_term2 == "1":
                search_by_authors()
                break

            elif search_term2 == "2":
                search_by_title()
                break

            elif search_term2 == "3":
                print('''\n\033[1mPlease take note that the database only contains papers published from 1995 to 2021.\033[0m''')
                while True:
                    print('''\033[1mHow would you like to search by Year? Please enter the corresponding number:\033[0m\n
        Papers of exact year :                       1
        Papers published in certain range of years:  2\n''')
                    search_year = input("Enter your choice: ")
                    if search_year == "1":
                        search_papers_by_year()
                        break
                    elif search_year == "2":
                        search_papers_by_range()
                        break
                    else:
                        print("\nYou have entered an unexpected input. Please try again.\n")
                break

            elif search_term2 == "4":   
                search_by_journal()
                break

            elif search_term2 == "5":
                search_cowritten_papers()
                break

            elif search_term2 == "6":
                search_paper_ID()
                break
            
            elif search_term2 == "7":
                search_gen()
                break
                
            else:
                print("\nYou have entered an unexpected input. Please try again.")

        return
    
    this_is_the_search_engine()
    f1.close()  
    return
# ----------------------------------------------------------------------------------




# -------------------------------- Retrieval engine --------------------------------
def final_retrieval_engine():
    # import and open the dataset
    f1 = open("master.json", encoding = "utf - 8")
    v1 = json.loads(f1.read())
    
    
    def limit_authorfullname():
        # ---------- print Paper ID and author full names in table format ----------
        print()
        print("{:<15}{:<100}".format('Paper ID','Author(s) Full Name'))     # header
        
        count = 0
        flag = "Y"
        for papers in v1:
            allauthorlist = []
            for authors in papers["authors"]:                               # join author first and last names
                fullname = f'{authors["fname"]} {authors["lname"]}'
                allauthorlist.append(fullname)
                authorfullname = ", ".join(allauthorlist)
            print(f'{str(papers["paper_id"]) : <13}| {textwrap.fill(text=authorfullname, width = 100).replace(nl, nl15)}')           
        
        # ---------------- limit to only 10 results shown at a time ----------------
            count = count + 1
            if count >= 10:
                flag = "Red"
                while flag == "Red":

                    view = input(f"\tType 'Y' to view more or type 'N' to stop: ")
                    if view == "Y" or view == "y":
                        flag = "Y"
                        count = 0
                        break

                    elif view == "N" or view == "n":
                        flag = "N"
                        break

                    else:
                        print("\nYou have entered an unexpected input. Please try again.")

                if flag == "N":
                    print("\nRetrieval complete.")
                    break
                    return


    def limit_numauthors_lname():
        # ---- print Paper ID, number of authors and last names in table format ----
        print()
        print("{:<15}{:<25}{:<80}".format('Paper ID','No. of Authors','Author(s) Last Name'))
        
        count = 0
        for papers in v1:
            all_authors = []
            i=0
            for authors in papers["authors"]:
                all_authors.append(authors["lname"])
                i=i+1
                allauthorname = ", ".join(all_authors)
            print(f'{str(papers["paper_id"]) : <13}| {str(i)  : <23}| {textwrap.fill(text=allauthorname, width = 80).replace(nl, nl40)}')
        
        # ---------------- limit to only 10 results shown at a time ----------------
            count = count + 1
            if count >= 10:
                flag = "Red"
                while flag == "Red":

                    view = input(f"\tType 'Y' to view more or type 'N' to stop: ")
                    if view == "Y" or view == "y":
                        flag = "Y"
                        count = 0
                        break

                    elif view == "N" or view == "n":
                        flag = "N"
                        break

                    else:
                        print("\nYou have entered an unexpected input. Please try again.")

                if flag == "N":
                    print("\nRetrieval complete.")
                    break
                    return

                
    def limit_keywords():
        # --------------- print Paper ID and keywords in table format --------------
        print()
        print("{:<15}{:<100}".format('Paper ID','Keywords'))
        
        count = 0
        for papers in v1:
            keyword_list = []
            if len(papers["paper"]["keywords"])>0:
                Tkeyword = []
                for words in papers["paper"]["keywords"]:
                    Tkeyword.append(words.title())
            else:
                Tkeyword = papers["paper"]["keywords"]
            keywords = ", ".join(Tkeyword)
            keyword_list.append(keywords)

            for keywords in keyword_list:
                if keyword_list[0] != '':
                    print(f'{str(papers["paper_id"]) : <13}| {textwrap.fill(text=keyword_list[0], width = 100).replace(nl, nl15)}')
                else:
                    print(f'{str(papers["paper_id"]) : <13}| No keywords found')
        
        # ---------------- limit to only 10 results shown at a time ----------------
            count = count + 1
            if count >= 10:
                flag = "Red"
                while flag == "Red":

                    view = input(f"\tType 'Y' to view more or type 'N' to stop: ")
                    if view == "Y" or view == "y":
                        flag = "Y"
                        count = 0
                        break

                    elif view == "N" or view == "n":
                        flag = "N"
                        break

                    else:
                        print("\nYou have entered an unexpected input. Please try again.")

                if flag == "N":
                    print("\nRetrieval complete.")
                    break
                    return

                
    def limit_cauthors():
        # ------- print Paper ID, contact author(s) and email in table format ------
        print()
        print("{:<15}{:<25}{:<80}".format('Paper ID','Contact Author Name','Contact Author Email'))
        
        count = 0
        for papers in v1:
            found = False
            for no_allauthors in range(len(papers["authors"])):
                try:
                    if papers["authors"][no_allauthors]["cauthor"] == True:
                        found = True
                        cauthor = papers['authors'][no_allauthors]['fname'] + " " + papers['authors'][no_allauthors]['lname']
                        print(f"{str(papers['paper_id']) : <13}| {cauthor : <23}| {papers['authors'][no_allauthors]['email']}")
                except KeyError:
                    print(f"{str(papers['paper_id']) : <13}| {cauthor : <23}| Email not found.")
                    
         # ---------------- limit to only 10 results shown at a time ----------------
            count = count + 1
            if count >= 10:
                flag = "Red"
                while flag == "Red":

                    view = input(f"\tType 'Y' to view more or type 'N' to stop: ")
                    if view == "Y" or view == "y":
                        flag = "Y"
                        count = 0
                        break

                    elif view == "N" or view == "n":
                        flag = "N"
                        break

                    else:
                        print("\nYou have entered an unexpected input. Please try again.")

                if flag == "N":
                    print("\nRetrieval complete.")
                    break
                    return

                
    def limit_titles():
        # ------------- print Paper ID and paper title in table format ------------
        print()
        print("{:<15}{:<100}".format('Paper ID','Paper Title'))
        
        count = 0
        for papers in v1:
            print(f'{str(papers["paper_id"]) : <13}| {textwrap.fill(text=papers["paper"]["title"].title(), width = 100).replace(nl, nl15)}')
        
        # ---------------- limit to only 10 results shown at a time ----------------
            count = count + 1
            if count >= 10:
                flag = "Red"
                while flag == "Red":

                    view = input(f"\tType 'Y' to view more or type 'N' to stop: ")
                    if view == "Y" or view == "y":
                        flag = "Y"
                        count = 0
                        break

                    elif view == "N" or view == "n":
                        flag = "N"
                        break

                    else:
                        print("\nYou have entered an unexpected input. Please try again.")

                if flag == "N":
                    print("\nRetrieval complete.")
                    break
                    return

                
    def limit_printall():
        # -------------------- print all details in table format -------------------
        # this was the one of the first sections we did, so it's quite messy
        print()
        
        count = 0
        for papers in v1:
            
            # Paper ID
            print(f"{'Paper ID' : <10}| {papers['paper_id']}")
            
            # Authors
            smalllistauthor = []
            for everyauthor in papers["authors"]:
                fullname = f"{everyauthor['fname']} {everyauthor['lname']}"
                smalllistauthor.append(fullname)
            author = ", ".join(smalllistauthor)
            print(f"{'Authors' : <10}| {textwrap.fill(text=author, width = 100).replace(nl, nl10)}")
            
            # Title
            print(f"{'Title' : <10}| {textwrap.fill(text=papers['paper']['title'].title(), width = 100).replace(nl, nl10)}")
            
            # Year
            print(f"{'Year' : <10}| {papers['year']}")
            
            # Journal
            print(f"{'Journal' : <10}| {papers['source']['journal']}")                

            # cauthor and email
            cauthors = []
            cauthors_email = []    
            for no_allauthors in range(len(papers["authors"])):
                try:
                    if papers["authors"][no_allauthors]["cauthor"] == True:
                        cauthor = papers['authors'][no_allauthors]['fname'] + " " + papers['authors'][no_allauthors]['lname']
                        cauthors.append(cauthor)
                        cauthors_email.append(papers['authors'][no_allauthors]['email'])
                except KeyError:
                    pass
            if cauthors == []:
                cauthors.append("No contact author found")
                cauthors_email.append("No contact email found")
            cauthors_s = ', '.join(cauthors)
            cauthors_email_s = ', '.join(cauthors_email)
            print(f"{'CAuthor' : <10}| {textwrap.fill(text=cauthors_s, width = 100).replace(nl, nl10)}")
            print(f"{'Email' : <10}| {textwrap.fill(text=cauthors_email_s, width = 100).replace(nl, nl10)}")

            # keywords
            if len(papers["paper"]["keywords"])>0:
                Tkeyword = []
                for words in papers["paper"]["keywords"]:
                    Tkeyword.append(words.title())
                keyword_list = ", ".join(Tkeyword)
                print(f"{'Keywords' : <10}| {textwrap.fill(text=keyword_list, width = 100).replace(nl, nl10)}")
            else:
                print(f"{'Keywords' : <10}| No keywords found")

            # volume
            try:
                if papers['source']['volume'] == [] or papers['source']['volume'] == False:
                    print(f"{'Volume' : <10}| Volume not found.")
                else:
                    print(f"{'Volume' : <10}| {papers['source']['volume']}")
            except KeyError:
                print(f"{'Volume' : <10}| Volume not found")

            # number
            try:
                if papers['source']['number'] == [] or papers['source']['number'] == False:
                    print(f"{'Number' : <10}| Number not found.")
                else:
                    if type(papers['source']['number']) == list:
                        print(f"{'Number' : <10}| {', '.join(map(str, papers['source']['number']))}")
                    else:
                        print(f"{'Number' : <10}| {papers['source']['number']}")
            except KeyError:
                print(f"{'Number' : <10}| Number not found.")

            # pages
            print(f"{'Pages' : <10}| {papers['source']['pages']['s_page']}-{papers['source']['pages']['e_page']}\n")
              
        # ---------------- limit to only 10 results shown at a time ----------------
            count = count + 1
            flag = 'Red'                             
            if count >= 10:
                while flag == 'Red':

                    view = input(f"\tType 'Y' to view more or type 'N' to stop: ")
                    if view == "Y" or view == "y":
                        flag = "Y"
                        count = 0
                        break

                    elif view == "N" or view == "n":
                        flag = "N"
                        break
                    else:
                        print("\nYou have entered an unexpected input. Please try again.")

            if flag == "N":
                print("\nRetrieval complete.")
                break
                return

            
    def this_is_retrieve_engine():  
    # --------------------- Combine all the retrieval functions ---------------------
        print('''\033[1m
What would you like to retrieve?
Please enter the corresponding number:\033[0m''')
        print('''
    Name of authors:   1
    Number of authors: 2
    Keywords:          3
    Contact authors:   4
    Titles:            5
    All info:          6
            ''')

        while True:
            search_term = input("Enter search term: ")

            if search_term == "1":
                limit_authorfullname()
                break

            elif search_term == "2":
                limit_numauthors_lname()
                break

            elif search_term == "3":
                limit_keywords()
                break
                
            elif search_term == "4":
                limit_cauthors()
                break
                
            elif search_term == "5":
                limit_titles()
                break
                
            elif search_term == "6":
                limit_printall()
                break

            else:
                print("\nYou have entered an unexpected input. Please try again.") 
    
    this_is_retrieve_engine()
    f1.close()  
    return
# ----------------------------------------------------------------------------------




# ---------------------- Combine Search and Retrieve engines -----------------------    
def final_of_final(): 
    # ----- power "on", program won't be terminated until user turns it "off" ------
    power = "on"
    while power == "on":
        print('''\033[1mThis is a program that allows search and retrieval of Wee Kim Wee School of Communication and Information academic journals.\nWhat would you like to do? Please enter the corresponding number:\033[0m\n
        Search:    1
        Retrieve:  2''')

        while True:
            final_input = input("\nEnter: ")

            if final_input == "1":
                final_search_engine()
                break

            elif final_input == "2":
                final_retrieval_engine()
                break

            else:
                print("\nYou have entered an unexpected input. Please try again.")

    # ------ after first use, ask if user wants to continue using the program ------
    # if user wants to continue using, power remains on
    # if user wants to terminate the programe, power off
        while True:
            cont = input("\nWould you like to continue using the program? (Y/N) ")
            if cont.lower() == "y":
                print()
                break
            elif cont.lower() == "n":
                power = "off"
                break
            else:
                print("\nYou have entered an unexpected input. Please try again.")
        
    # final message before power off
    print("\nThank you for using this program.")
    return
# ----------------------------------------------------------------------------------





# --------------------------------- Lock fuction ----------------------------------- 
if __name__ == "__main__":    
    final_of_final()
# ----------------------------------------------------------------------------------

This is a program that allows search and retrieval of Wee Kim Wee School of Communication and Information academic journals.
What would you like to do? Please enter the corresponding number:

        Search:    1
        Retrieve:  2

Enter: 

You have entered an unexpected input. Please try again.

Enter: 1

What would you like to search?
Please enter the corresponding number:

    Authors:            1
    Title:              2
    Year:               3
    Journal:            4
    Co-written papers:  5
    Paper ID:           6
    General:            7
            

Enter search term: 1

Enter the name of author to start search.
Enter author name: 1

No matching papers found for search term '1'.

Search complete.

Would you like to continue using the program? (Y/N) n

Thank you for using this program.
